# Semantic Chunking RAG

本notebook旨在演示如何通过 **语义分割（Semantic Chunking）** 技术来优化检索增强生成（RAG）流程。


### 传统分割方法的局限性
传统的文本分割方法，如固定大小分割，常常会粗暴地切断句子或语义完整的段落，导致上下文信息丢失，从而影响检索的准确性和最终答案的质量。

### 为什么使用语义分割？
**语义分割**是一种更智能的文本分块策略。它不依赖于固定的字符数，而是通过分析句子之间的**语义相似度**来决定分割点。当相邻句子之间的语义差异较大时，意味着一个主题可能已经结束，新的主题即将开始，这便是一个理想的分割点。

### Semantic Chunking RAG流程
![](figures/02%20Semantic%20Chunking%20RAG/流程图1.png)
![](figures/02%20Semantic%20Chunking%20RAG/流程图2.png)

简单来说，Semantic Chunking的过程，就是判断两个**相邻句子**之间是否存在**语义关联**，而语义关联是通过**嵌入向量**的相似度判断的。

Semantic Chunking的核心思路就是：

若相邻句子的嵌入向量相似度更高，那么就认为这两句话在讲同一件事情，它们应当处在同一个文本块中。

In [ ]:
import regex as re
import numpy as np
from rag_evaluate import Embedder, LLMJudge, load_data, ReplyModel
import warnings
import torch
warnings.filterwarnings('ignore')  # 忽略所有的警告

加载评估的相关数据。

In [2]:
documents, QA = load_data(data_dir='data')

### 定义语义分割（Semantic Chunking）函数

这是我们流程的核心。`semantic_chunk_documents` 函数实现了将长文档智能地分割成语义连贯的文本块的逻辑。其工作流程如下：

1.  **分割成句子**: 使用正则表达式将整篇文档分割成独立的句子。这是语义分析的基本单位。
2.  **生成句子嵌入**: 调用 `embedder` 将每个句子转换成一个高维向量。这个向量代表了句子的语义信息。
3.  **计算相邻距离**: 遍历所有相邻的句子对，计算它们嵌入向量之间的**余弦距离**（1 - 余弦相似度）。距离越大，表示两个句子的语义差异越大。
4.  **确定分割阈值**: 我们不使用固定的距离阈值，而是计算所有相邻距离的**百分位数**（例如95%）。这意味着只有当两个句子的语义差异大于数据集中95%的句子对时，我们才认为这里应该是一个分割点。这种自适应的阈值比硬编码更鲁棒。
5.  **执行分割**: 再次遍历句子，如果一个句子与其后继者的距离超过了我们计算出的阈值，我们就在此处创建一个新的文本块（Chunk）。否则，将后继句子加入到当前块中。
6.  **整合与清理**: 将所有生成的块汇总，并清理GPU缓存以释放内存。

In [3]:
def semantic_chunk_documents(documents, embedder, percentile_threshold=95):
    """
    对每个文档进行语义分割。
    """
    # 
    doc_chunks = []
    # 遍历每个文档
    for di, document in enumerate(documents):
        print(f"开始处理文档{di + 1}")
        # 1. 将文本块分割成句子
        sentences = re.split(r'(?<=[。！？\n])', document)
        sentences = [s.strip() for s in sentences if s.strip()]
        print(f"分割得到{len(sentences)}个句子")
    
        # 2. 为每个句子生成嵌入
        embeddings = embedder.embed(sentences)
    
        # 3. 计算相邻句子的余弦距离
        distances = []
        for i in range(len(embeddings) - 1):
            similarity = embedder.similarity([embeddings[i]], [embeddings[i + 1]])[0][0]
            distance = 1 - similarity
            distances.append(distance)
    
        # 4. 根据百分位数确定分割阈值
        breakpoint_distance_threshold = np.percentile(distances, percentile_threshold)
        print(f"分割阈值: {breakpoint_distance_threshold}")
    
        # 5. 根据阈值进行分割
        chunks = []
        current_chunk_sentences = [sentences[0]]
        for i in range(len(distances)):
            # 如果两句之间的距离大于阈值
            if distances[i] > breakpoint_distance_threshold:
                chunks.append("\n".join(current_chunk_sentences))
                current_chunk_sentences = [sentences[i+1]]
            else:
                current_chunk_sentences.append(sentences[i+1])
        
        # 添加最后一个块
        chunks.append("\n".join(current_chunk_sentences))
        print(f"共得到{len(chunks)}个文本块")
        doc_chunks.extend(chunks)
    
        del embeddings
        torch.cuda.empty_cache()
    
    return doc_chunks

#### 执行语义分割

现在，我们调用刚刚定义的 `semantic_chunk_documents` 函数来处理我们加载的文档。

这个过程会遍历每个文档，执行上述的分割逻辑，并将所有文档产生的小块（chunks）合并到一个列表中。最后，我们打印出总的块数和前几个块的内容，以直观地感受一下分割的效果。

In [4]:
# 用作语义分割的嵌入模型
embedder = Embedder()

# 执行结构化分割
chunks = semantic_chunk_documents(documents, embedder)

print(f"--- 语义分割完成，共 {len(chunks)} 个块 ---")
# 查看前4个块
for i, chunk in enumerate(chunks[:4]):
    print(f"【块 {i+1}】:\n{chunk}\n")

开始处理文档1
分割得到386个句子
分割阈值: 0.7188476562499999
共得到21个文本块
开始处理文档2
分割得到220个句子
分割阈值: 0.673828125
共得到12个文本块
开始处理文档3
分割得到504个句子
分割阈值: 0.77962646484375
共得到27个文本块
--- 语义分割完成，共 60 个块 ---
【块 1】:
# DeepSeek-R1：通过强化学习激发大语言模型的推理能力
## DeepSeek-AI
research@deepseek.com
### 摘要
我们介绍了第一代推理模型DeepSeek-R1-Zero和DeepSeek-R1。
DeepSeek-R1-Zero是通过大规模强化学习（RL）训练的模型，无需将监督微调（SFT）作为初步步骤，展现出显著的推理能力。
通过强化学习，DeepSeek-R1-Zero自然涌现出许多强大且有趣的推理行为。

【块 2】:
然而，它面临着可读性差和语言混合等挑战。

【块 3】:
为解决这些问题并进一步提升推理性能，我们引入了DeepSeek-R1，该模型在强化学习之前融入了多阶段训练和冷启动数据。
DeepSeek-R1在推理任务上的性能可与OpenAI-o1-1217相媲美。
为支持研究社区，我们开源了DeepSeek-R1-Zero、DeepSeek-R1，以及基于Qwen和Llama从DeepSeek-R1蒸馏得到的六个稠密模型（1.5B、7B、8B、14B、32B、70B）。
|模型|GPQA Diamond（单次通过率）|SWE-bench Verified（已解决）|AIME 2024（单次通过率）|MMLU（百分位）|MATH-500（单次通过率）|Codeforces（单次通过率）|
|----|----|----|----|----|----|----|
|DeepSeek-R1|71.5|72.6|79.8|90.8|97.3|96.3|
|OpenAI-o1-1217|75.7|84.8|79.2|91.8|96.4|96.6|
|DeepSeek-R1-32B|62.1|59.1|79.8|90.8|97.3|96.3|
|OpenAI-o1-mini|60.0|60.0|63.6|85.2|90.0|93.4|
|DeepSeek-V3

### 向量化：为文本块和问题创建嵌入

在这里，我们执行两个向量化任务：
1.  **向量化所有文本块**: 我们将上一步中通过语义分割得到的所有 `chunks` 逐一转换成嵌入向量。这些向量构成了我们可供检索的知识库。
2.  **向量化所有问题**: 同样地，我们将 `QA` 数据集中的所有 `queries`（问题）也转换成嵌入向量。

这样，问题和知识库中的文本块就被置于同一个高维语义空间中，我们可以通过计算它们向量之间的距离来判断其相关性。

In [9]:
# 加载问题与正确答案
queries = [qa["question"] for qa in QA['data']]
answers_gt = [qa["answer"] for qa in QA['data']]

# 对所有的文本块向量化
chunk_embeddings = []
for i, chunk in enumerate(chunks):
    print(f"\r正在向量化文本块 {i+1}/{len(chunks)}", end='')
    chunk_embedding = embedder.embed(chunk)
    chunk_embeddings.append(chunk_embedding)
    
# 对所有的问题向量化
query_embeddings = embedder.embed(queries)

正在向量化文本块 60/60

### 检索（Retrieval）：查找最相关的文本块
这是 RAG 中的 "R"（Retrieval）环节。我们的目标是，对于每一个问题，从我们庞大的知识库（`chunks`）中找出最可能包含答案的几个文本块。

过程如下：
1.  **计算相似度**: 我们使用 `embedder.similarity` 函数计算每个问题向量与所有文本块向量之间的余弦相似度。
2.  **获取 Top-K**: 对于每个问题，我们从相似度得分中选出最高的 `k` 个文本块。在这里，我们设置 `k=3`，意味着为每个问题检索3个最相关的上下文片段。
3.  **展示结果**: 我们打印出每个问题及其检索到的 Top-K 文本块和对应的相似度分数，以便直观地检查检索质量。一个好的检索结果应该在语义上与问题高度相关。

In [17]:
similarity = embedder.similarity(query_embeddings, chunk_embeddings)
k = 3
top_k_indices = embedder.get_top_k(similarity, k)
for i, query in enumerate(queries[:1]):
    retrieved_chunks = []
    print(f"问题：{query}")
    print("-" * 30)
    for doc_idx in top_k_indices[i]:
        print(f"文本块(相似度{similarity[i, doc_idx]:.4f})：\n{chunks[doc_idx][:100]}......")
        print("=" * 30)

问题：在后训练过程中，为什么Qwen3模型在经过“思考模式融合”和“通用RL”阶段后，在AIME’24和LiveCodeBench等具有挑战性的任务上性能反而有所下降？
------------------------------
文本块(相似度0.8228)：
#### 思考模式融合和通用RL的效果
为了评估后训练期间思考模式融合和通用强化学习（RL）的有效性，我们对Qwen-32B模型的各个阶段进行了评估。
除了前面提到的数据集，我们还引入了几个内部基准来......
文本块(相似度0.7271)：
# Qwen3技术报告
Qwen团队
https://huggingface.co/Qwen
https://modelscope.cn/organization/qwen
https://githu......
文本块(相似度0.7197)：
#### 聊天模板设计
为了更好地集成两种模式并使用户能够动态切换模型的思考过程，我们为Qwen3设计了聊天模板，如表9所示。
具体而言，对于思考模式和非思考模式的样本，我们分别在用户查询或系统消息中......


### 生成（Generation）：基于检索到的内容回答问题

这是 RAG 中的 "G"（Generation）环节。现在我们已经为每个问题找到了相关的上下文信息，下一步是利用这些信息来生成一个流畅、准确的答案。

我们执行以下步骤：
1.  **实例化生成模型**: 我们加载 `ReplyModel`，它封装了一个大型语言模型（LLM）。
2.  **构建提示（Prompt）**: 对于每个问题，我们将问题本身和上一步检索到的 `retrieved_chunks` 一起打包，形成一个完整的提示。这个提示会明确指示 LLM：“请根据以下上下文信息来回答这个问题”。
3.  **生成答案**: 我们将构建好的提示发送给 LLM，并获取其生成的答案。

通过提供明确的上下文，我们引导 LLM 基于我们提供的事实进行回答，而不是依赖其内部可能过时或不相关的知识，从而大大减少“幻觉”现象。

In [11]:
llm = ReplyModel()
answers = []
for i, query in enumerate(queries):
    retrieved_chunks = []
    for doc_idx in top_k_indices[i]:
        retrieved_chunks.append(chunks[doc_idx])
        
    answer = llm.answer(query, retrieved_chunks)
    answers.append(answer)
    print(query)
    print(answer)
    print("=" * 30)

在后训练过程中，为什么Qwen3模型在经过“思考模式融合”和“通用RL”阶段后，在AIME’24和LiveCodeBench等具有挑战性的任务上性能反而有所下降？
根据提供的文档，Qwen3模型在后训练过程中，在AIME’24和LiveCodeBench等具有挑战性的任务上性能下降的原因是由于模型在更广泛的通用任务上训练，这可能损害了其处理复杂问题的专业能力。文档中提到，虽然思考模式融合和通用RL在许多基准测试中带来了提升，但在这两个特定任务上，性能实际上有所下降。推测这种退化是由于模型在更广泛的通用任务上训练，导致其专业能力被稀释。因此，开发团队选择接受这种性能权衡，以增强模型的整体多功能性。
在针对轻量级模型的“强到弱蒸馏”管道中，在线蒸馏（Online Distillation）阶段的具体实现方式是什么？
根据提供的文档，在线蒸馏（Online Distillation）阶段的具体实现方式是：从教师logits蒸馏使学生模型能够扩展其探索空间并增强其推理潜力。在实验中，通过比较蒸馏后的性能和计算成本，发现在线蒸馏比强化学习取得了显著更好的性能，同时仅需要约1/10的GPU小时。此外，在线蒸馏后的模型在AIME’24和AIME’25基准上的pass@64分数与初始检查点相比有所提高，而强化学习并未导致pass@64分数的任何改进。这表明在线蒸馏通过利用更强的教师模型来指导学生模型学习，从而提升了模型的性能。
Qwen3的开发团队在预训练数据筛选和模型后训练阶段，具体实施了哪些伦理审查（Ethical Review）流程来确保模型的安全性并减少偏见？
根据提供的文档，Qwen3的开发团队在预训练数据筛选和模型后训练阶段实施了以下伦理审查（Ethical Review）流程来确保模型的安全性并减少偏见：

1. **多语言数据注释系统**：开发了一个多语言数据注释系统，对超过30万亿token进行了多个维度的注释，如教育价值、领域、域和安全性。这有助于提高训练数据的质量和多样性，并支持更有效的数据过滤和组合。

2. **数据过滤过程**：在预训练数据的构建过程中，采用了严格的两阶段过滤过程：查询过滤和响应过滤。查询过滤阶段使用Qwen2.5-72B-Instruct来识别和删除不易验证的查询，包括包含多个子问题或请求一般文本生成的查询。此外，排除了无需使用CoT推

最后，我们对Semantic Chunking RAG进行评估：

In [12]:
judge = LLMJudge()
judge.evaluate(queries, answers_gt, answers)

1 正在评估问题：在后训练过程中，为什么Qwen3模型在经过“思考模式融合”和“通用RL”阶段后，在AIME’24和LiveCodeBench等具有挑战性的任务上性能反而有所下降？
--------------------
{'scores': {'Correctness': 5, 'Completeness': 3, 'Clarity & Conciseness': 5}, 'reasoning': '生成答案准确捕捉了正确答案的核心因果关系（通用任务训练稀释专业能力）和团队权衡决策，语义等价性完整。但在完整性上遗漏了正确答案中关于『其他基准测试表现提升』这一关键对比信息，导致关键信息点缺失。语言表达简洁流畅，逻辑清晰，无冗余或歧义。', 'final_score': 4.333333333333333}
2 正在评估问题：在针对轻量级模型的“强到弱蒸馏”管道中，在线蒸馏（Online Distillation）阶段的具体实现方式是什么？
--------------------
{'scores': {'Correctness': 3, 'Completeness': 2, 'Clarity & Conciseness': 4}, 'reasoning': '生成答案描述了在线蒸馏中学生模型如何通过logits对齐进行微调，这是在线蒸馏的一部分内容，因此在正确性上部分符合。然而，它遗漏了正确答案中的关键点，如在线蒸馏在性能提升、计算成本降低以及与强化学习的对比结果等核心优势，完整性较差。语言表达较为清晰，结构合理，没有明显冗余或混乱之处，因此清晰简洁性得分较高。', 'final_score': 3.0}
3 正在评估问题：Qwen3的开发团队在预训练数据筛选和模型后训练阶段，具体实施了哪些伦理审查（Ethical Review）流程来确保模型的安全性并减少偏见？
--------------------
{'scores': {'Correctness': 1, 'Completeness': 1, 'Clarity & Conciseness': 5}, 'reasoning': '生成答案未能提供任何正确的信息，与正确答案完全不符，导致正确性和完整性得分极低。然而，语言表达简洁清晰，没有冗余或结构问题，符合高清晰度和简洁性标准。', 'final_scor

得到最终的评分：3.43
从结果来看，Semantic Chunking RAG的评分与标准的RAG几乎一致，没有带来明显的性能提升。
这可能是因为在Semantic Chunking时，有时会导致文本块的长度太长，从而在生成阶段无法得到高质量的回复。